# AGE DETECTION ON IMDB-WIKI DATASET 

Import the necessary libraries

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from PIL import Image

Load the dataset

In [2]:
data_dir='IMDB-Wiki'

Data Preprocessing

In [3]:
def calculate_age_from_filename(filename):
    try:        
        parts = filename.split('_')
        birthdate_str = parts[2]  
        photo_year = int(parts[3].split('.')[0]) 
        
        birthdate_parts = birthdate_str.split('-')
        birth_year = int(birthdate_parts[0])
        birth_month = int(birthdate_parts[1]) if birthdate_parts[1] != '0' else 1
        birth_day = int(birthdate_parts[2]) if birthdate_parts[2] != '0' else 1

        age = photo_year - birth_year
        return age
    except Exception as e:
        print(f"Error processing file {filename}: {e}")
        return None

def load_data(data_dir):
    images = []
    ages = []

    for dir_name, _, filenames in os.walk(data_dir):
        for filename in filenames:
            img_path = os.path.join(dir_name, filename)
            age = calculate_age_from_filename(filename)
            if age is not None:
                try:
                    img = Image.open(img_path)
                    img.verify() 
                    img = Image.open(img_path).convert('RGB')
                    img = img.resize((224, 224))  
                    img = np.array(img) / 255.0  
                    images.append(img)
                    ages.append(age)
                except Exception as e:
                    print(f"Error processing file {filename}: {e}")
                    continue

    return np.array(images, dtype='float32'), np.array(ages, dtype='float32')

images, ages = load_data(data_dir)

dataset = tf.data.Dataset.from_tensor_slices((images, ages))

for image, age in dataset.take(5):
    print(f"Image shape: {image.shape}, Age: {age.numpy()}")

tf.data.experimental.save(dataset, 'age_detection_dataset')

Image shape: (224, 224, 3), Age: 37.0
Image shape: (224, 224, 3), Age: 33.0
Image shape: (224, 224, 3), Age: 28.0
Image shape: (224, 224, 3), Age: 46.0
Image shape: (224, 224, 3), Age: 44.0
Instructions for updating:
Use `tf.data.Dataset.save(...)` instead.


Split the dataset into training and validation sets

In [4]:
x_train, x_val, y_train, y_val = train_test_split(images, ages, test_size=0.2, random_state=42)

Load the VGG16 model

In [5]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1)(x)  

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

In [6]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])

In [7]:
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=20, batch_size=32)

Epoch 1/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 622s 6s/step - loss: 373.0916 - mae: 14.6384 - val_loss: 192.5018 - val_mae: 9.8997
Epoch 2/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 611s 6s/step - loss: 219.8428 - mae: 11.4036 - val_loss: 296.1147 - val_mae: 12.9689
Epoch 3/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 629s 6s/step - loss: 231.8870 - mae: 11.6003 - val_loss: 242.0675 - val_mae: 11.4974
Epoch 4/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 595s 6s/step - loss: 170.3378 - mae: 9.9981 - val_loss: 268.8062 - val_mae: 12.4156
Epoch 5/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 636s 6s/step - loss: 167.7003 - mae: 10.0691 - val_loss: 192.3312 - val_mae: 10.0768
Epoch 6/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 637s 6s/step - loss: 159.8135 - mae: 9.6937 - val_loss: 220.0211 - val_mae: 10.8979
Epoch 7/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 626s 6s/step - loss: 145.1538 - mae: 9.3074 - val_loss: 270.7570 - val_mae: 12.5738
Epoch 8/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 605s 6s/step - loss: 146.7985 - mae: 9.3479 - val_loss: 540.9284 - val_mae: 19.9320
Epoch

Save the model

In [8]:
model.save('age_detection_model.h5')

In [9]:
test_loss, test_mae = model.evaluate(x_val, y_val)
print(f'Test MAE: {test_mae}')

27/27 ━━━━━━━━━━━━━━━━━━━━ 115s 4s/step - loss: 269.2838 - mae: 12.7688
Test MAE: 12.444560050964355


GUI

In [10]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf

Load the model

In [11]:
model = tf.keras.models.load_model('age_detection_model.h5')

In [12]:
# Function to predict age
def predict_age(image_path):
    try:
        img = Image.open(image_path).convert('RGB')
        img = img.resize((224, 224))  
        img_array = np.array(img) / 255.0  
        img_array = np.expand_dims(img_array, axis=0)  

        # Predict age
        predicted_age = model.predict(img_array)
        return int(predicted_age[0][0])
    except Exception as e:
        messagebox.showerror("Error", f"Error in predicting age: {e}")
        return None

In [13]:
# Function to open image file
def open_file():
    file_path = filedialog.askopenfilename(
        title="Select Image",
        filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")]
    )
    if file_path:
        load_image(file_path)

In [14]:
# Function to load and display image
def load_image(image_path):
    try:
        img = Image.open(image_path)
        img.thumbnail((300, 300))  
        
        img_tk = ImageTk.PhotoImage(img)
        panel.configure(image=img_tk)
        panel.image = img_tk  

        # Predict age
        predicted_age = predict_age(image_path)
        if predicted_age is not None:
            age_label.config(text=f"Predicted Age: {predicted_age}")
    except Exception as e:
        messagebox.showerror("Error", f"Error loading image: {e}")


In [15]:
# Create the main window
root = tk.Tk()
root.title("Age Detection")

panel = tk.Label(root)
panel.pack(padx=10, pady=10)

age_label = tk.Label(root, text="Predicted Age: ", font=("Helvetica", 14))
age_label.pack(pady=20)

open_button = tk.Button(root, text="Upload Image", command=open_file)
open_button.pack(pady=10)

root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 500ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 358ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step
